In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [108]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [109]:
from nltk.corpus import stopwords

In [110]:
import tensorflow as tf

In [111]:
df_train = pd.read_csv('/content/twitter_training.csv')
df_test= pd.read_csv('/content/twitter_validation.csv')

In [112]:
df_train.columns = ['tweet_id', 'entity', 'sentiment', 'tweet_content']
df_test.columns = ['tweet_id', 'entity', 'sentiment', 'tweet_content']

In [113]:
df_test.head()

,tweet_id,entity,sentiment,tweet_content
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [114]:
df_test.isnull().sum()

,0
tweet_id,0
entity,0
sentiment,0
tweet_content,0


In [115]:
df_train.dropna(axis=0,inplace=True)

In [116]:
df_train['tweet_id'].nunique()

12447

In [117]:
df_train=df_train.loc[:10000]

In [118]:
df_train.shape

(9889, 4)

In [119]:
import re

In [120]:
def preprocess(text):
  text = re.sub(r'http\S+|www.\S+', '', text)
    # Remove mentions and hashtags
  text = re.sub(r'@\w+|#\w+', '', text)
    # Remove special characters and numbers
  text = re.sub(r'[^A-Za-z\s]', '', text)
    # Convert to lowercase
  text = text.lower()
    # Remove extra whitespace
  text = re.sub(r'\s+', ' ', text).strip()
  return text

In [121]:
df_train['tweet_content']=df_train['tweet_content'].apply(preprocess)
df_test['tweet_content']=df_test['tweet_content'].apply(preprocess)

In [122]:
df_train.head()

,tweet_id,entity,sentiment,tweet_content
0,2401,Borderlands,Positive,i am coming to the borders and i will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you all
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [123]:
df_test.shape

(999, 4)

In [124]:
from sklearn.feature_extraction.text import CountVectorizer

In [125]:
vectorizer = CountVectorizer()

In [126]:
X_train = vectorizer.fit_transform(df_train['tweet_content'])
X_test = vectorizer.transform(df_test['tweet_content'])

In [127]:
from sklearn.ensemble import RandomForestClassifier

In [128]:
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

In [129]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [130]:
y_train = le.fit_transform(df_train['sentiment'])
y_test = le.fit_transform(df_test['sentiment'])

In [131]:
rfc.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

In [132]:
y_pred = rfc.predict(X_test)

In [133]:
from sklearn.metrics import classification_report,confusion_matrix

In [134]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.26      0.35      0.30       125
           1       0.51      0.51      0.51       265
           2       0.30      0.64      0.41       134
           3       0.77      0.45      0.57       475

    accuracy                           0.48       999
   macro avg       0.46      0.49      0.45       999
weighted avg       0.57      0.48      0.50       999



In [135]:
print(confusion_matrix(y_pred,y_test))

[[ 44  25  33  23]
 [ 49 135  50  31]
 [ 15  23  86  10]
 [ 63  83 116 213]]
